<a href="https://colab.research.google.com/github/TanyaSharma14/harm-classification/blob/main/Copy_of_MinorProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section

In [ ]:
!pip install torch transformers pandas scikit-learn youtube-transcript-api beautifulsoup4 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 13.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/Harmful.csv")

df = df[['transcript']].dropna()
df['label'] = 1

safe_text = [
    "this video discusses current events",
    "general discussion and opinions",
    "educational tutorial video",
    "news analysis and reporting",
    "public awareness content",
    "interview and discussion",
    "informative session for viewers",
    "general knowledge video",
    "technology explanation",
    "social media discussion"
] * 1000

safe_df = pd.DataFrame({
    "transcript": safe_text,
    "label": 0
})

df = pd.concat([df, safe_df], ignore_index=True)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["transcript"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_enc = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
test_enc = tokenizer(test_texts, truncation=True, padding=True, max_length=256)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_ds = Dataset(train_enc, train_labels)
test_ds = Dataset(test_enc, test_labels)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

args = TrainingArguments(
    output_dir="bert_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds
)

trainer.train()

model.save_pretrained("bert_model")
tokenizer.save_pretrained("bert_model")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss
100,0.032400
200,0.000200
300,0.028300
400,0.000100
500,0.000000
600,0.000000
700,0.000000
800,0.000000
900,0.000000
1000,0.000000


('bert_model/tokenizer_config.json',
 'bert_model/special_tokens_map.json',
 'bert_model/vocab.txt',
 'bert_model/added_tokens.json')

In [ ]:
import torch
import requests
from transformers import BertTokenizer, BertForSequenceClassification
from youtube_transcript_api import YouTubeTranscriptApi
from bs4 import BeautifulSoup

tokenizer = BertTokenizer.from_pretrained("bert_model")
model = BertForSequenceClassification.from_pretrained("bert_model")
model.eval()

with open("/content/en.txt", encoding="utf-8", errors="ignore") as f:
    bad_words = [w.strip().lower() for w in f if w.strip()]

def extract_video_id(url):
    if "youtu.be/" in url:


        return url.split("youtu.be/")[1].split("?")[0]
    if "v=" in url:
        return url.split("v=")[1].split("&")[0]
    if "/live/" in url:
        return url.split("/live/")[1].split("?")[0]
    raise ValueError("Invalid YouTube URL")

def get_text(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join(x["text"] for x in transcript), "Transcript"
    except:
        html = requests.get(f"https://www.youtube.com/watch?v={video_id}").text
        soup = BeautifulSoup(html, "html.parser")
        title = soup.title.text if soup.title else ""
        desc = soup.find("meta", {"name": "description"})
        return title + " " + (desc["content"] if desc else ""), "Title + Description"

def find_bad_words(text):
    t = text.lower()
    return list(set([w for w in bad_words if w in t]))

url = input("Enter YouTube video URL: ")
video_id = extract_video_id(url)

text, source = get_text(video_id)

inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)

with torch.no_grad():
    output = model(**inputs)
    probs = torch.softmax(output.logits, dim=1)
    bert_score = probs[0][1].item()

hits = find_bad_words(text)

severity = min(bert_score + 0.04 * len(hits), 1.0)

if severity < 0.3:
    label = "NOT HARMFUL"
elif severity < 0.6:
    label = "MODERATE"
else:
    label = "HARMFUL"

print("\n========== RESULT ==========")
print("Text Source:", source)
print("Final Classification:", label)
print("Severity Score:", round(severity, 2))
print("Detected Harmful Words:", hits[:15])


Enter YouTube video URL: https://youtu.be/AOhlCPi7J2M?si=BahhWylXxgkOHnkS

========== RESULT ==========
Text Source: Title + Description
Final Classification: HARMFUL
Severity Score: 1.0
Detected Harmful Words: []


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
SAVE_PATH = "/content/drive/MyDrive/youtube_harmful_content_filtering/bert_model"


In [ ]:
import os
os.makedirs(SAVE_PATH, exist_ok=True)


In [ ]:
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)


('/content/drive/MyDrive/youtube_harmful_content_filtering/bert_model/tokenizer_config.json',
 '/content/drive/MyDrive/youtube_harmful_content_filtering/bert_model/special_tokens_map.json',
 '/content/drive/MyDrive/youtube_harmful_content_filtering/bert_model/vocab.txt',
 '/content/drive/MyDrive/youtube_harmful_content_filtering/bert_model/added_tokens.json')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained(SAVE_PATH)
model = BertForSequenceClassification.from_pretrained(SAVE_PATH)
